In [28]:

import pandas as pd
import pybaseball
pybaseball.cache.enable()
print(pd.__version__)
from pybaseball import statcast
balldata = statcast(start_dt='2025-01-01',end_dt='2025-07-30')
balldata = pd.DataFrame(balldata)
from sklearn.model_selection import train_test_split as tts
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix as cm, classification_report as cr
from collections import Counter

2.3.1
This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 138/138 [00:07<00:00, 19.56it/s]
/sciclone/home/tdfelton/.conda/envs/baseball2/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


In [32]:
# STEP 0: Initial filtering (only regular season games)
balldata = balldata[balldata['game_type'] == 'R'].copy()

# STEP 1: Sort to maintain proper pitch order within at-bats
balldata.sort_values(by=['game_pk', 'at_bat_number', 'pitch_number'], inplace=True)
balldata.reset_index(drop=True, inplace=True)

# STEP 2: Label whether pitch is final pitch of at-bat (Stage 1 target)
balldata['stage1_label'] = (
    balldata.groupby(['game_pk', 'at_bat_number'])['pitch_number'].transform('max') == balldata['pitch_number']
).astype(int)

# STEP 3: Add sequencing features (last 3 pitch types, plate_x, spin rate)
def add_last_n_features(df, col, n=3):
    for i in range(1, n+1):
        df[f'{col}_prev{i}'] = df.groupby(['game_pk', 'at_bat_number'])[col].shift(i)
    return df

balldata = add_last_n_features(balldata, 'pitch_type', 7)
balldata = add_last_n_features(balldata, 'plate_x', 7)
balldata = add_last_n_features(balldata, 'release_spin_rate', 7)
balldata = add_last_n_features(balldata, 'vx0', 7)
balldata = add_last_n_features(balldata, 'plate_z', 7)
balldata = add_last_n_features(balldata, 'vy0', 7)
balldata = add_last_n_features(balldata, 'ax', 7)
balldata = add_last_n_features(balldata, 'release_extension', 7)
balldata = add_last_n_features(balldata, 'az', 7)
balldata = add_last_n_features(balldata, 'release_speed', 7)
balldata = add_last_n_features(balldata, 'release_pos_z', 7)
balldata = add_last_n_features(balldata, 'api_break_x_arm', 7)
balldata = add_last_n_features(balldata, 'effective_speed', 7)
balldata = add_last_n_features(balldata, 'release_pos_y', 7)
balldata = add_last_n_features(balldata, 'spin_axis', 7)
balldata = add_last_n_features(balldata, 'api_break_z_with_gravity', 7)
balldata = add_last_n_features(balldata, 'arm_angle', 7)
balldata = add_last_n_features(balldata, 'release_pos_x', 7)
balldata = add_last_n_features(balldata, 'api_break_x_batter_in', 7)
balldata = add_last_n_features(balldata, 'pfx_z', 7)



# STEP 4: Extract date parts
balldata['game_year'] = pd.to_datetime(balldata['game_date']).dt.year
balldata['game_month'] = pd.to_datetime(balldata['game_date']).dt.month
balldata['game_day'] = pd.to_datetime(balldata['game_date']).dt.day

# STEP 5: Drop unneeded columns
cols_to_drop = [
    'type', 'player_name', 'des', 'home_team', 'game_year', 'away_team', 'game_type',
    'woba_value', 'woba_denom', 'babip_value', 'iso_value', 'launch_speed_angle', 'hyper_speed',
    'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score', 'bat_win_dif',
    'delta_home_win_exp', 'delta_run_exp', 'delta_pitcher_run_exp', 'bat_score', 'fld_score',
    'home_score_diff', 'home_win_exp', 'on_3b', 'on_2b', 'on_1b', 'pitch_name', 'home_score', 'away_score', 'description',
    'hit_location', 'bb_type', 'hc_x', 'hc_y', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'batter',
    'stand', 'game_pk', 'bat_speed', 'swing_length', 'bat_score_diff', 'bat_win_exp', 'attack_angle', 'attack_direction', 'swing_path_tilt',
    'age_bat', 'zone', 'intercept_ball_minus_batter_pos_x_inches', 'intercept_ball_minus_batter_pos_y_inches'
]

X_stage1 = balldata.drop(columns=['events'], errors='ignore')
X_stage1 = X_stage1.drop(columns=[col for col in X_stage1.columns if 'depreciated' in col or 'legacy' in col or 'estimated' in col])
X_stage1 = X_stage1.drop(columns=[col for col in cols_to_drop if col in X_stage1.columns])

# STEP 6: Set up categorical features for CatBoost
categorical_cols = [
    'game_month', 'pitch_type', 'pitcher', 'p_throws',
    'balls', 'strikes', 'inning', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
    'fielder_7', 'fielder_8', 'fielder_9', 'pitch_number', 'at_bat_number',
    'if_fielding_alignment', 'of_fielding_alignment', 'age_pit', 'n_thruorder_pitcher',
    'n_priorpa_thisgame_player_at_bat', 'pitcher_days_since_prev_game', 'batter_days_since_prev_game',
    'pitcher_days_until_next_game', 'batter_days_until_next_game', 'inning_topbot',
    'pitch_type_prev1', 'pitch_type_prev2', 'pitch_type_prev3'
]
pitch_type_prev_cols = [col for col in X_stage1.columns if col.startswith('pitch_type_prev')]
categorical_cols = list(set(categorical_cols + pitch_type_prev_cols))

X_stage1[categorical_cols] = X_stage1[categorical_cols].astype(str)

# STEP 7: Assign target
y_stage1 = balldata['stage1_label']
X_stage1 = X_stage1.drop(columns='stage1_label')
X_stage1[categorical_cols] = X_stage1[categorical_cols].fillna("MISSING")
X_stage1.fillna(-999, inplace=True)


The below code is kind of useless unless I'm doing live analysis, did it more so to practice on a more complete dataset before moving to the full batting average predictor

In [37]:
x_train1, x_test1, y_train1, y_test1 = tts(X_stage1, y_stage1, test_size=0.2, stratify=y_stage1)

#class_counts = Counter(y_train1)
#total = sum(class_counts.values())
#class_weights = {cls: total/count for cls, count in class_counts.items()}
#print("Class weights:", class_weights)

model_stage1 = CatBoostClassifier(
    verbose=100,
    scale_pos_weight=2.0,
    eval_metric='F1',
)

model_stage1.fit(x_train1, y_train1, eval_set = (x_test1, y_test1), early_stopping_rounds = 50, cat_features=categorical_cols)

y_pred1 = model_stage1.predict(x_test1)
print(cm(y_test1, y_pred1))
print(cr(y_test1, y_pred1, digits=3))

Learning rate set to 0.137153
0:	learn: 0.5472333	test: 0.5438585	best: 0.5438585 (0)	total: 123ms	remaining: 2m 2s
100:	learn: 0.7724221	test: 0.7900633	best: 0.7900633 (100)	total: 11.5s	remaining: 1m 42s
200:	learn: 0.8520639	test: 0.8743535	best: 0.8745904 (197)	total: 27.8s	remaining: 1m 50s
300:	learn: 0.8649816	test: 0.8777034	best: 0.8780159 (276)	total: 42s	remaining: 1m 37s
400:	learn: 0.8773245	test: 0.8879906	best: 0.8879906 (400)	total: 55.5s	remaining: 1m 22s
500:	learn: 0.8840293	test: 0.8909537	best: 0.8910215 (490)	total: 1m 8s	remaining: 1m 8s
600:	learn: 0.8890127	test: 0.8924643	best: 0.8927311 (571)	total: 1m 21s	remaining: 54.2s
700:	learn: 0.8951175	test: 0.8945632	best: 0.8946305 (698)	total: 1m 35s	remaining: 40.6s
800:	learn: 0.8978037	test: 0.8953556	best: 0.8953641 (796)	total: 1m 47s	remaining: 26.6s
900:	learn: 0.9024919	test: 0.8960442	best: 0.8963485 (862)	total: 2m 1s	remaining: 13.3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8

In [39]:
final_pitches = balldata[balldata['stage1_label'] == 1].copy()
hit_events = ['single', 'double', 'triple', 'home_run']
final_pitches['stage2_label'] = final_pitches['events'].isin(hit_events).astype(int)
X_stage2 = final_pitches.drop(columns=['stage2_label', 'events'], errors='ignore')
X_stage2 = X_stage2.drop(columns=[col for col in X_stage2.columns if 'depreciated' in col or 'legacy' in col or 'estimated' in col])
X_stage2 = X_stage2.drop(columns=[col for col in cols_to_drop if col in X_stage2.columns])

X_stage2[categorical_cols] = X_stage2[categorical_cols].astype(str)
X_stage2[categorical_cols] = X_stage2[categorical_cols].fillna("MISSING")
X_stage2.fillna(-999, inplace=True)

y_stage2 = final_pitches['stage2_label']

In [43]:
x_train2, x_test2, y_train2, y_test2 = tts(X_stage2, y_stage2, test_size=0.2, stratify=y_stage2)

model_stage2 = CatBoostClassifier(
    verbose=100,
    scale_pos_weight=(len(y_train2) - sum(y_train2)) / sum(y_train2),
    eval_metric='F1',
    early_stopping_rounds=50
)

model_stage2.fit(x_train2, y_train2, cat_features=categorical_cols)

y_pred2 = model_stage2.predict(x_test2)
print(cm(y_test2, y_pred2))
print(cr(y_test2, y_pred2, digits=3))


Learning rate set to 0.072787
0:	learn: 0.6194472	total: 36.4ms	remaining: 36.3s
100:	learn: 0.6766492	total: 3.82s	remaining: 34s
200:	learn: 0.6958912	total: 7.84s	remaining: 31.1s
300:	learn: 0.7150288	total: 11.8s	remaining: 27.3s
400:	learn: 0.7298968	total: 15.9s	remaining: 23.7s
500:	learn: 0.7410379	total: 19.9s	remaining: 19.8s
600:	learn: 0.7505275	total: 24s	remaining: 16s
700:	learn: 0.7573101	total: 28.5s	remaining: 12.2s
800:	learn: 0.7651665	total: 32.9s	remaining: 8.16s
900:	learn: 0.7732695	total: 37s	remaining: 4.06s
999:	learn: 0.7807931	total: 41s	remaining: 0us
[[13458  5547]
 [ 2262  3085]]
              precision    recall  f1-score   support

           0      0.856     0.708     0.775     19005
           1      0.357     0.577     0.441      5347

    accuracy                          0.679     24352
   macro avg      0.607     0.643     0.608     24352
weighted avg      0.747     0.679     0.702     24352



In [44]:
importances = model_stage2.get_feature_importance()
features = X_stage2.columns
imp_df = pd.DataFrame({ "Feature": features, "Importance": importances })

# Step 3: Select top N features
top_features = imp_df.sort_values(by="Importance", ascending=False)["Feature"].head(50).tolist()
cat_top = [col for col in top_features if col in categorical_cols]

# Step 4: Subset and retrain with only top features
X_top = X_stage2[top_features]
x_train2, x_test2, y_train2, y_test2 = tts(X_top, y_stage2, test_size=0.2, stratify=y_stage2)

filtered_model = CatBoostClassifier(
    verbose=100,
    scale_pos_weight=(len(y_train2) - sum(y_train2)) / sum(y_train2),
    eval_metric='F1',
    early_stopping_rounds=50
)
filtered_model.fit(x_train2, y_train2, cat_features=cat_top)

# Step 5: Evaluate
y_pred2 = filtered_model.predict(x_test2)
print(cm(y_test2, y_pred2))
print(cr(y_test2, y_pred2, digits=3))

Learning rate set to 0.072787
0:	learn: 0.6282841	total: 22.1ms	remaining: 22s
100:	learn: 0.6769420	total: 2.47s	remaining: 22s
200:	learn: 0.7011304	total: 5.2s	remaining: 20.7s
300:	learn: 0.7175848	total: 7.94s	remaining: 18.4s
400:	learn: 0.7251174	total: 11s	remaining: 16.4s
500:	learn: 0.7345328	total: 13.9s	remaining: 13.9s
600:	learn: 0.7426887	total: 16.8s	remaining: 11.1s
700:	learn: 0.7502184	total: 19.7s	remaining: 8.4s
800:	learn: 0.7572378	total: 22.5s	remaining: 5.59s
900:	learn: 0.7631761	total: 25.4s	remaining: 2.79s
999:	learn: 0.7710239	total: 28.2s	remaining: 0us
[[13467  5538]
 [ 2223  3124]]
              precision    recall  f1-score   support

           0      0.858     0.709     0.776     19005
           1      0.361     0.584     0.446      5347

    accuracy                          0.681     24352
   macro avg      0.609     0.646     0.611     24352
weighted avg      0.749     0.681     0.704     24352

